In [30]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import geopandas as gpd

from dowser.config import (
    RAW_WPDX,
    INTERIM_BOUNDARIES,
    INTERIM_POINTS_TZA,
    INTERIM_POINTS_KEN,
    TABLES,
    PROCESSED,
)
from dowser.data_io import read_adm_any, read_wpdx_csv
from dowser.geo import spatial_join_points
from dowser.summaries import quality_report, admin_counts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# ---------------------------
# A) Load full dataset
# ---------------------------
wpdx_full_path = RAW_WPDX / "kenia+tanzania_wpdx.csv"
wpdx_full = pd.read_csv(wpdx_full_path, low_memory=False)

print("Totale righe:", len(wpdx_full))
print("Colonne:", wpdx_full.columns.tolist()[:20])  # prime 20

# ---------------------------
# B) Split by country and convert to GeoDataFrame
# ---------------------------

# Filtra per paese
tza_df = wpdx_full[wpdx_full["#clean_country_name"].str.lower().str.contains("tanzania", na=False)].copy()
ken_df = wpdx_full[wpdx_full["#clean_country_name"].str.lower().str.contains("kenya", na=False)].copy()

# Rimuovi righe senza coordinate valide
tza_df = tza_df[tza_df["#lon_deg"].notna() & tza_df["#lat_deg"].notna()]
ken_df = ken_df[ken_df["#lon_deg"].notna() & ken_df["#lat_deg"].notna()]

# Converti in GeoDataFrame
tza_wp = gpd.GeoDataFrame(
    tza_df,
    geometry=gpd.points_from_xy(tza_df["#lon_deg"], tza_df["#lat_deg"]),
    crs="EPSG:4326"
)

ken_wp = gpd.GeoDataFrame(
    ken_df,
    geometry=gpd.points_from_xy(ken_df["#lon_deg"], ken_df["#lat_deg"]),
    crs="EPSG:4326"
)

print("\nTanzania punti:", len(tza_wp))
print("Kenya punti:", len(ken_wp))

# ---------------------------
# C) Check status distribution (per label)
# ---------------------------
print("\n--- Status Tanzania ---")
print(tza_wp["#status_clean"].value_counts())

print("\n--- Status Kenya ---")
print(ken_wp["#status_clean"].value_counts())

# ---------------------------
# D) Save full datasets
# ---------------------------
tza_wp.to_parquet(INTERIM_POINTS_TZA / "tza_wpdx_all.parquet", index=False)
ken_wp.to_parquet(INTERIM_POINTS_KEN / "ken_wpdx_all.parquet", index=False)

print("\nSaved:")
print(" -", INTERIM_POINTS_TZA / "tza_wpdx_all.parquet")
print(" -", INTERIM_POINTS_KEN / "ken_wpdx_all.parquet")

Totale righe: 39471
Colonne: ['row_id', '#source', '#lat_deg', '#lon_deg', '#report_date', '#status_id', '#water_source_clean', '#water_source_category', '#water_tech_clean', '#water_tech_category', '#facility_type', '#clean_country_name', '#clean_country_id', '#clean_adm1', '#clean_adm2', '#clean_adm3', '#clean_adm4', '#install_year', '#installer', '#rehab_year']

Tanzania punti: 17518
Kenya punti: 21953

--- Status Tanzania ---
#status_clean
Non-Functional                8640
Non-Functional, dry season    5137
Functional                    2978
Functional, needs repair       375
Abandoned/Decommissioned       328
Others                          44
Functional, not in use          16
Name: count, dtype: int64

--- Status Kenya ---
#status_clean
Non-Functional                13578
Functional, not in use         4757
Functional                     1964
Functional, needs repair       1461
Non-Functional, dry season      190
Abandoned/Decommissioned          3
Name: count, dtype: int64

Sa

In [23]:
# ---------------------------
# E) Create binary labels
# ---------------------------

def map_status_to_label(status):
    if pd.isna(status):
        return None
    status_lower = status.lower()
    if "functional" in status_lower and "non" not in status_lower:
        return 1  # Functional
    elif "non-functional" in status_lower or "abandoned" in status_lower:
        return 0  # Non-Functional
    else:
        return None  # Exclude (Others)

tza_wp["label"] = tza_wp["#status_clean"].apply(map_status_to_label)
ken_wp["label"] = ken_wp["#status_clean"].apply(map_status_to_label)

# Rimuovi righe senza label
tza_wp = tza_wp[tza_wp["label"].notna()].copy()
ken_wp = ken_wp[ken_wp["label"].notna()].copy()

print("--- Tanzania ---")
print(f"Totale: {len(tza_wp)}")
print(tza_wp["label"].value_counts())
print(f"% Functional: {tza_wp['label'].mean()*100:.1f}%")

print("\n--- Kenya ---")
print(f"Totale: {len(ken_wp)}")
print(ken_wp["label"].value_counts())
print(f"% Functional: {ken_wp['label'].mean()*100:.1f}%")

# Salva con label
tza_wp.to_parquet(INTERIM_POINTS_TZA / "tza_wpdx_labeled.parquet", index=False)
ken_wp.to_parquet(INTERIM_POINTS_KEN / "ken_wpdx_labeled.parquet", index=False)

print("\nSaved labeled datasets!")

--- Tanzania ---
Totale: 17474
label
0.0    14105
1.0     3369
Name: count, dtype: int64
% Functional: 19.3%

--- Kenya ---
Totale: 21953
label
0    13771
1     8182
Name: count, dtype: int64
% Functional: 37.3%

Saved labeled datasets!


In [24]:
# Cerca colonne relative a profondità, yield, etc.
wpdx_full = pd.read_csv(RAW_WPDX / "kenia+tanzania_wpdx.csv", low_memory=False)

print("Tutte le colonne:")
for col in sorted(wpdx_full.columns):
    print(f"  {col}")

# Cerca specificamente depth, yield, water level
keywords = ["depth", "yield", "level", "static", "dynamic", "water_table", "drill", "bore"]
relevant_cols = [c for c in wpdx_full.columns if any(k in c.lower() for k in keywords)]
print("\n--- Colonne potenzialmente rilevanti ---")
print(relevant_cols)

# Check anche nei valori (a volte depth è in "notes")
print("\n--- Sample notes ---")
if "#notes" in wpdx_full.columns:
    print(wpdx_full["#notes"].dropna().head(20))

Tutte le colonne:
  #activity_id
  #adm1
  #adm2
  #adm3
  #clean_adm1
  #clean_adm2
  #clean_adm3
  #clean_adm4
  #clean_country_id
  #clean_country_name
  #country_id
  #country_name
  #data_lnk
  #distance_to_city
  #distance_to_primary_road
  #distance_to_secondary_road
  #distance_to_tertiary_road
  #distance_to_town
  #facility_type
  #fecal_coliform_presence
  #fecal_coliform_value
  #install_year
  #installer
  #lat_deg
  #lon_deg
  #management
  #management_clean
  #notes
  #orig_lnk
  #pay
  #pay_clean
  #photo_lnk
  #pop_who_would_gain_access
  #rehab_year
  #rehabilitator
  #report_date
  #scheme_id
  #source
  #status
  #status_clean
  #status_id
  #subjective_quality
  #subjective_quality_clean
  #water_source
  #water_source_category
  #water_source_clean
  #water_tech
  #water_tech_category
  #water_tech_clean
  #wpdx_id
  New Georeferenced Column 
  cluster_size
  converted
  count
  created_timestamp
  crucialness_score
  dataset_title
  days_since_report
  is_duplica

In [25]:
# Intanto verifichiamo altri campi potenzialmente utili in WPDx
print("--- Water Source (tipo di fonte) ---")
print(wpdx_full["#water_source_clean"].value_counts().head(15))

print("\n--- Water Tech (tecnologia estrazione) ---")
print(wpdx_full["#water_tech_clean"].value_counts().head(15))

print("\n--- Facility Type ---")
print(wpdx_full["#facility_type"].value_counts())

--- Water Source (tipo di fonte) ---
#water_source_clean
Protected Well             17655
Borehole/Tubewell          12487
Protected Spring            5175
Undefined Well              1366
Sand or Sub-surface Dam      953
Rainwater Harvesting         850
Piped Water                  834
Delivered Water               13
Unprotected Well               7
Undefined Spring               1
Packaged Water                 1
Name: count, dtype: int64

--- Water Tech (tecnologia estrazione) ---
#water_tech_clean
Public Tapstand              11241
Motorized Pump - Electric     6836
Hand Pump - Nira              3709
Hand Pump                     2450
Hand Pump - Mono              2031
Hand Pump - SWN 80            1853
Hand Pump - Afridev           1753
Hand Pump - Rope              1099
Hand Pump - India Mark         986
Motorized Pump                 312
Rope and Bucket                 81
Hand Pump - Tara                63
Hand Pump - Blue                54
Hand Pump - Climax              52
Ha

In [26]:
# Filtriamo solo Borehole
borehole_df = wpdx_full[wpdx_full["#water_source_clean"] == "Borehole/Tubewell"].copy()

print(f"Totale Borehole: {len(borehole_df)}")
print("\n--- Status dei Borehole ---")
print(borehole_df["#status_clean"].value_counts())

# Crea label per Task A
def label_borehole(status):
    if pd.isna(status):
        return None
    status_lower = status.lower()
    if "functional" in status_lower and "non" not in status_lower:
        return 1  # Acqua trovata
    elif "non-functional" in status_lower or "abandoned" in status_lower:
        return 0  # Acqua non trovata (o insufficiente)
    else:
        return None

borehole_df["label"] = borehole_df["#status_clean"].apply(label_borehole)
borehole_df = borehole_df[borehole_df["label"].notna()]

print(f"\n--- Dataset finale Task A ---")
print(f"Totale: {len(borehole_df)}")
print(borehole_df["label"].value_counts())
print(f"% Successo: {borehole_df['label'].mean()*100:.1f}%")

# Split per paese
print("\n--- Per paese ---")
for country in ["Tanzania", "Kenya"]:
    mask = borehole_df["#clean_country_name"].str.contains(country, case=False, na=False)
    subset = borehole_df[mask]
    print(f"{country}: {len(subset)} borehole, {subset['label'].mean()*100:.1f}% successo")


Totale Borehole: 12487

--- Status dei Borehole ---
#status_clean
Non-Functional                7521
Functional                    1641
Non-Functional, dry season    1568
Functional, not in use        1414
Functional, needs repair       251
Abandoned/Decommissioned        74
Others                          18
Name: count, dtype: int64

--- Dataset finale Task A ---
Totale: 12469
label
0.0    9163
1.0    3306
Name: count, dtype: int64
% Successo: 26.5%

--- Per paese ---
Tanzania: 6045 borehole, 17.0% successo
Kenya: 6424 borehole, 35.4% successo


In [27]:
# Filtriamo Protected Well
well_df = wpdx_full[wpdx_full["#water_source_clean"] == "Protected Well"].copy()

print(f"Totale Protected Well: {len(well_df)}")
print("\n--- Status dei Well ---")
print(well_df["#status_clean"].value_counts())

# Crea label
well_df["label"] = well_df["#status_clean"].apply(label_borehole)
well_df = well_df[well_df["label"].notna()]

print(f"\n--- Dataset Well ---")
print(f"Totale: {len(well_df)}")
print(well_df["label"].value_counts())
print(f"% Successo: {well_df['label'].mean()*100:.1f}%")

# Confronto
print("\n" + "="*50)
print("CONFRONTO BOREHOLE vs WELL")
print("="*50)
print(f"Borehole: {len(borehole_df)} punti, {borehole_df['label'].mean()*100:.1f}% successo")
print(f"Well:     {len(well_df)} punti, {well_df['label'].mean()*100:.1f}% successo")

Totale Protected Well: 17655

--- Status dei Well ---
#status_clean
Non-Functional                11151
Non-Functional, dry season     2113
Functional                     1871
Functional, not in use         1538
Functional, needs repair        864
Abandoned/Decommissioned         96
Others                           22
Name: count, dtype: int64

--- Dataset Well ---
Totale: 17633
label
0.0    13360
1.0     4273
Name: count, dtype: int64
% Successo: 24.2%

CONFRONTO BOREHOLE vs WELL
Borehole: 12469 punti, 26.5% successo
Well:     17633 punti, 24.2% successo


In [28]:
# ---------------------------
# Verifica Spring
# ---------------------------
spring_df = wpdx_full[wpdx_full["#water_source_clean"] == "Protected Spring"].copy()

print(f"Totale Protected Spring: {len(spring_df)}")
print("\n--- Status delle Spring ---")
print(spring_df["#status_clean"].value_counts())

spring_df["label"] = spring_df["#status_clean"].apply(label_borehole)
spring_df = spring_df[spring_df["label"].notna()]

print(f"\n--- Dataset Spring ---")
print(f"Totale: {len(spring_df)}")
print(spring_df["label"].value_counts())
print(f"% Successo: {spring_df['label'].mean()*100:.1f}%")

# ---------------------------
# RIEPILOGO TOTALE
# ---------------------------
print("\n" + "="*60)
print("RIEPILOGO DATASET MULTI-CLASS")
print("="*60)
print(f"{'Tipo':<20} {'Totale':>10} {'Success':>10} {'Fail':>10} {'%Succ':>10}")
print("-"*60)
print(f"{'Borehole (30-100m)':<20} {len(borehole_df):>10} {int(borehole_df['label'].sum()):>10} {int((1-borehole_df['label']).sum()):>10} {borehole_df['label'].mean()*100:>9.1f}%")
print(f"{'Well (5-20m)':<20} {len(well_df):>10} {int(well_df['label'].sum()):>10} {int((1-well_df['label']).sum()):>10} {well_df['label'].mean()*100:>9.1f}%")
print(f"{'Spring (0-5m)':<20} {len(spring_df):>10} {int(spring_df['label'].sum()):>10} {int((1-spring_df['label']).sum()):>10} {spring_df['label'].mean()*100:>9.1f}%")

Totale Protected Spring: 5175

--- Status delle Spring ---
#status_clean
Non-Functional                2630
Non-Functional, dry season    1261
Functional                    1019
Abandoned/Decommissioned       142
Functional, needs repair        63
Functional, not in use          56
Others                           4
Name: count, dtype: int64

--- Dataset Spring ---
Totale: 5171
label
0.0    4033
1.0    1138
Name: count, dtype: int64
% Successo: 22.0%

RIEPILOGO DATASET MULTI-CLASS
Tipo                     Totale    Success       Fail      %Succ
------------------------------------------------------------
Borehole (30-100m)        12469       3306       9163      26.5%
Well (5-20m)              17633       4273      13360      24.2%
Spring (0-5m)              5171       1138       4033      22.0%


In [31]:
# ---------------------------
# Crea dataset multi-class
# ---------------------------

# Prepara i tre dataframe
borehole_df = wpdx_full[wpdx_full["#water_source_clean"] == "Borehole/Tubewell"].copy()
well_df = wpdx_full[wpdx_full["#water_source_clean"] == "Protected Well"].copy()
spring_df = wpdx_full[wpdx_full["#water_source_clean"] == "Protected Spring"].copy()

def get_success(status):
    if pd.isna(status):
        return None
    status_lower = status.lower()
    if "functional" in status_lower and "non" not in status_lower:
        return True
    elif "non-functional" in status_lower or "abandoned" in status_lower:
        return False
    return None

# Applica e filtra
for df in [borehole_df, well_df, spring_df]:
    df["success"] = df["#status_clean"].apply(get_success)

borehole_df = borehole_df[borehole_df["success"].notna()].copy()
well_df = well_df[well_df["success"].notna()].copy()
spring_df = spring_df[spring_df["success"].notna()].copy()

# Assegna classi
# Classe 0: NO_WATER (fallimento di qualsiasi tipo)
# Classe 1: SURFACE - Spring success (0-5m)
# Classe 2: SHALLOW - Well success (5-20m)
# Classe 3: DEEP - Borehole success (30-100m)

borehole_df["class"] = borehole_df["success"].apply(lambda x: 3 if x else 0)
well_df["class"] = well_df["success"].apply(lambda x: 2 if x else 0)
spring_df["class"] = spring_df["success"].apply(lambda x: 1 if x else 0)

# Aggiungi info sul tipo di fonte originale
borehole_df["source_type"] = "borehole"
well_df["source_type"] = "well"
spring_df["source_type"] = "spring"

# Combina tutto
combined_df = pd.concat([borehole_df, well_df, spring_df], ignore_index=True)

# Converti in GeoDataFrame
combined_gdf = gpd.GeoDataFrame(
    combined_df,
    geometry=gpd.points_from_xy(combined_df["#lon_deg"], combined_df["#lat_deg"]),
    crs="EPSG:4326"
)

# Aggiungi paese
combined_gdf["country"] = combined_gdf["#clean_country_name"].apply(
    lambda x: "TZA" if "tanzania" in str(x).lower() else "KEN"
)

# ---------------------------
# Riepilogo classi
# ---------------------------
print("="*60)
print("DATASET MULTI-CLASS FINALE")
print("="*60)

class_names = {0: "NO_WATER", 1: "SURFACE (0-5m)", 2: "SHALLOW (5-20m)", 3: "DEEP (30-100m)"}
print(f"\n{'Classe':<25} {'Punti':>10} {'%':>10}")
print("-"*45)
for cls in [0, 1, 2, 3]:
    count = (combined_gdf["class"] == cls).sum()
    pct = count / len(combined_gdf) * 100
    print(f"{class_names[cls]:<25} {count:>10} {pct:>9.1f}%")
print("-"*45)
print(f"{'TOTALE':<25} {len(combined_gdf):>10}")

# ---------------------------
# Split per paese
# ---------------------------
print("\n--- Per paese ---")
for country in ["TZA", "KEN"]:
    subset = combined_gdf[combined_gdf["country"] == country]
    print(f"\n{country}: {len(subset)} punti")
    for cls in [0, 1, 2, 3]:
        count = (subset["class"] == cls).sum()
        print(f"  {class_names[cls]}: {count}")

# ---------------------------
# Salva
# ---------------------------
output_path = PROCESSED / "waterpoints_multiclass.parquet"
combined_gdf.to_parquet(output_path, index=False)
print(f"\n✅ Saved: {output_path}")

DATASET MULTI-CLASS FINALE

Classe                         Punti          %
---------------------------------------------
NO_WATER                       26556      75.3%
SURFACE (0-5m)                  1138       3.2%
SHALLOW (5-20m)                 4273      12.1%
DEEP (30-100m)                  3306       9.4%
---------------------------------------------
TOTALE                         35273

--- Per paese ---

TZA: 16958 punti
  NO_WATER: 13675
  SURFACE (0-5m): 1076
  SHALLOW (5-20m): 1178
  DEEP (30-100m): 1029

KEN: 18315 punti
  NO_WATER: 12881
  SURFACE (0-5m): 62
  SHALLOW (5-20m): 3095
  DEEP (30-100m): 2277

✅ Saved: /Users/leonardovannoli/work/dowser/dowser-v0/data/processed/waterpoints_multiclass.parquet
